# welcome 


### Data description 
### Main dataset: 
### This dataset informs the medical situation of individuals segregation the observations by municipality, serializing them and providing income data of each one of them. Also, it provides an specific score about overall health & the case of an specific condition.

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

url3 = r"Main_Dataset.csv"
md = pd.read_csv(url3)
url2 = r"Sociodemographics Data.csv"
sd = pd.read_csv(url2)
url1 = r"Municipality Pollution Data.csv"
mu = pd.read_csv(url1)


In [ ]:
filterd_values_md = md[md.isnull().any(axis=1)]
filterd_values_sd = sd[sd.isnull().any(axis=1)]
filterd_values_mu = mu[mu.isnull().any(axis=1)]

In [ ]:
# in case u want to always get the same sample use after the (n) after a coma random_state=: 
# This ensures that every time you run the code, you will get the same 500 rows sampled from your dataset (given the same input data).

md_AND_mu_merged = pd.merge(md, mu, on='Municipality')
md_r_sample = md_AND_mu_merged.sample(n=100,random_state=1)


fig = px.scatter_3d(md_r_sample, 
                     z='Overall_Health_Score', 
                     y='Traffic_Pollution', 
                     x='Air_Quality_Index',
                     color='Overall_Health_Score', 
                     opacity=0.65, 
                     size_max=20,
                     #hover_data=['Alcohol_Consumer','Smoker',"Physical_Activity",'Diet_Quality'],
                     #  color_discrete_sequence=px.colors.qualitative.Dark24
)

fig.update_layout(
    title="3D Scatter Plot of Health Score in relation with Traffic Pollution and AirQuality Index (Sampled Data)",
    scene=dict(
        yaxis_title="Traffic Pollution",
        zaxis_title="Health Score",
        xaxis_title=" AirQuality Index",
        bgcolor="black"  
    ),
    paper_bgcolor="black",  
    font=dict(color="white"),  
    scene_camera=dict(eye=dict(x=1.8, y=1.8, z=0.8))
)
fig.show()



### 1. **IMC > 30, Dieta poco saludable, No realiza actividad física**  
- Podría revelar una conexión clara con **diabetes tipo 2**, **síndrome metabólico** y **enfermedades articulares**.  
- **Idea:** Analizar si el sedentarismo y la dieta son los principales factores desencadenantes o si el IMC alto es el factor dominante.  



In [41]:

md_r_sample = md.sample(n=250,random_state=1).dropna(subset=["Income","Physical_Activity","BMI","Diet_Quality","Overall_Health_Score"])
fig = px.scatter(
    md_r_sample.query("(Physical_Activity == 'Low' | BMI > 30 | Diet_Quality == 'Poor')& Medical_Condition != 'Healthy'"), 
    x="BMI", 
    y="Overall_Health_Score", 
    color="Medical_Condition",
    hover_name="Medical_Condition", 
    size="BMI", 
    size_max=60,
    color_discrete_sequence=px.colors.diverging.Spectral
)

fig.show()


In [ ]:



random_states = [1, 10, 20, 30]  
fig = make_subplots(rows=1, cols=4, subplot_titles=[f"random sample = {rs}" for rs in random_states])

for i, rs in enumerate(random_states):
    md_r_sample = md.sample(n=1000, random_state=rs).dropna(subset=["Income", "Physical_Activity", "BMI", "Diet_Quality", "Overall_Health_Score"])
    filtered_data = md_r_sample.query("(Physical_Activity == 'Low' | BMI > 30 | Diet_Quality == 'Poor')& Medical_Condition != 'Healthy'")


    scatter = px.scatter(
    filtered_data, y="BMI", x="Overall_Health_Score",
    color="Medical_Condition",
    hover_name="Medical_Condition",
    size_max=10,  
    color_discrete_sequence=px.colors.diverging.Spectral
    )

    for trace in scatter.data:
        fig.add_trace(trace, row=1, col=i + 1)
fig.update_layout(title="Scatter Plots with Different Random States", 
    showlegend=False)


fig.show()



### 2. **Consumo excesivo de alcohol, Fumador, IMC > 25**  
- La combinación de tabaquismo y alcohol podría aumentar el riesgo de **enfermedades hepáticas** y **cáncer**.  
- **Idea:** Examinar si el consumo excesivo de alcohol o el tabaquismo tiene un impacto mayor en el desarrollo de cáncer.  

In [ ]:
Alcohol_Smoker_BMI_25 = md[(md["BMI"] > 25) & (md["Alcohol_Consumer"] == 1) & (md["Smoker"] == 1)]
Cancer = md[md["Medical_Condition"] == "Cancer"]


### 3. **Presión arterial alta, Fumador, Edad > 50**  
- Este grupo podría tener un riesgo elevado de **accidentes cerebrovasculares** y **enfermedades renales crónicas**.  
- **Idea:** Evaluar si la hipertensión o el tabaquismo tiene mayor influencia en la aparición de enfermedades renales.  


In [ ]:
md_AND_sd_merged = pd.merge(md, sd, on='ID')
Pressure_130_Smoker_Age_50 = md_AND_sd_merged[(md_AND_sd_merged["Blood_Pressure"] >= 130) & (md_AND_sd_merged["Age"]>= 50) & (md_AND_sd_merged["Smoker"]== 1)]

### 4. **Edad > 60, Consumo excesivo de alcohol, Enfermedades previas (como diabetes)**  
- El deterioro de la salud asociado con la edad y enfermedades preexistentes podría aumentar el riesgo de **insuficiencia hepática** y **demencia**.  
- **Idea:** Investigar si las enfermedades previas o el alcohol tienen un efecto mayor en el deterioro de la salud cognitiva.  


In [ ]:
md_AND_sd_merged = pd.merge(md, sd, on='ID')
Age_60_Alcohol_Ill = md_AND_sd_merged[(md_AND_sd_merged["Age"] >= 60) & (md_AND_sd_merged["Alcohol_Consumer"]==1)&(md_AND_sd_merged["Medical_Condition"]!="Healthy")]


### 5. **IMC > 30, Fumador, Presión arterial alta, No realiza actividad física**  
- Esta combinación podría relacionarse con **síndrome metabólico** y **enfermedades pulmonares** (como EPOC).  
- **Idea:** Evaluar cuál de los factores (IMC, tabaquismo o presión arterial) contribuye más a la aparición de enfermedades pulmonares.  

In [ ]:
BMI_smoker_Pressure_130 = md[(md["BMI"]>= 30) & (md["Blood_Pressure"] >= 130) & (md["Physical_Activity"]== "Low")]


### 6. **Sexo femenino, Edad > 50, Dieta poco saludable, IMC > 30**  
- Las mujeres postmenopáusicas con sobrepeso y una dieta poco saludable podrían tener un mayor riesgo de **osteoporosis** y **cáncer de mama**.  
- **Idea:** Examinar si la dieta o el IMC tiene un impacto más fuerte en la incidencia de cáncer o problemas óseos.  



### 7. **Presión arterial alta, Consumo de alcohol, Enfermedad previa (renal)**  
- Las personas con hipertensión y enfermedad renal previa que consumen alcohol podrían experimentar un deterioro rápido de la función renal.  
- **Idea:** Analizar si el alcohol o la hipertensión es el principal factor de deterioro renal.  



### 8. **Bajo puntaje de salud, Dieta poco saludable, No realiza actividad física**  
- Un bajo puntaje de salud podría reflejar un comportamiento de riesgo que desencadena **síndrome metabólico**, **diabetes** y **depresión**.  
- **Idea:** Evaluar si el puntaje de salud bajo refleja principalmente dieta y ejercicio o si hay otros factores psicosociales involucrados.  